### 실제 사이트 크롤링 - 네이버 환율

In [1]:
import bs4
import requests as rq
import pandas as pd

#### 사이트에 접속하고 탐색해 본다:

1. 먼저 다음 사이트에 들어가서 살펴본다:  [네이버 금융](https://finance.naver.com/marketindex/)
2. 크롬에서 3점 메뉴 버튼을 누르고 **도구 더보기** ==> **개발자 도구**를 선택해서 개발자 도구를 연다.
3. 개발자 도구 왼쪽 상단의 화살표 버튼을 눌러서 사이트 일부를 선택하고 해당 HTML코드를 볼 수 있다. "[CTRL] + [SHIFT] + C" 조합으로 대체할 수 있다.

In [2]:
# 웹사이트의 URL에 접속.
res = rq.get("https://finance.naver.com/marketindex")

In [3]:
# status_code가 200이면 OK.
# status_code가 4xx이면 접속 오류.
res.status_code

200

In [4]:
# 데이터 전체를 출력해 본다.
# 아직은 이해하기 어렵다.
print(res.text)              # print(res.content)와 비교.









<script language=javascript src="/template/head_js.nhn?referer=info.finance.naver.com&menu=marketindex&submenu=market"></script>
<script type="text/javascript" src="/js/info/jindo.min.ns.1.5.3.euckr.js"></script>
<script type="text/javascript" src="/js/jindo.1.5.3.element-text-patch.js"></script>


	
	
	<div id="container" style="padding-bottom:0px;">
	

		




<script language="JavaScript" src="/js/flashObject.js?20200107172122"></script>

























		<div class="market_include">
	<div class="market_data">
		<div class="market1">
			<div class="title">
				<h2 class="h_market1"><span>환전 고시 환율</span></h2>

			</div>
			<!-- data -->
			<div class="data">
				
				<ul class="data_lst" id="exchangeList">
				
				    
				         
				         
				         
				         				         			         
				        
					
				 	<li class="on">
					<a href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" class="head usd" onClick="clickcr(this, 'fr1.usdt', '', '',

In [5]:
my_html = res.text
soup = bs4.BeautifulSoup(my_html, 'html.parser')     # BeautifulSoup 객체 반환. 'html.parser' 사용.
print(soup.prettify())  

<script language="javascript" src="/template/head_js.nhn?referer=info.finance.naver.com&amp;menu=marketindex&amp;submenu=market">
</script>
<script src="/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript">
</script>
<script src="/js/jindo.1.5.3.element-text-patch.js" type="text/javascript">
</script>
<div id="container" style="padding-bottom:0px;">
 <script language="JavaScript" src="/js/flashObject.js?20200107172122">
 </script>
 <div class="market_include">
  <div class="market_data">
   <div class="market1">
    <div class="title">
     <h2 class="h_market1">
      <span>
       환전 고시 환율
      </span>
     </h2>
    </div>
    <!-- data -->
    <div class="data">
     <ul class="data_lst" id="exchangeList">
      <li class="on">
       <a class="head usd" href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
        <h3 class="h_lst">
         <span class="blind">
          미국 USD
         </span>
        </h3>
 

#### 환율 가져오기:

find_all() 메서드 사용.

In [6]:
res = soup.find_all('span', class_='value')                # span 태그 중 "value" 패턴 있는 클래스.
rate =[]
for i in range(4):
    rate += [float(''.join((res[i].text).split(',')))]     # 1000 단위 쉼표 "," 제거 처리.
result = {"외환":["USD","JPY","EUR","CNY"], "환율":rate}
df=pd.DataFrame(result)
df

,외환,환율
0,USD,1159.70
1,JPY,1062.04
2,EUR,1289.18
3,CNY,167.41


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
외환    4 non-null object
환율    4 non-null float64
dtypes: float64(1), object(1)
memory usage: 192.0+ bytes


select() 메서드 사용.

In [8]:
res = soup.select('span.value')                             # span 태그 중 "value" 패턴 있는 클래스.
rate =[]
for i in range(4):
    rate += [float(''.join((res[i].text).split(',')))]     # 1000 단위 쉼표 "," 제거 처리.
result = {"외환":["USD","JPY","EUR","CNY"], "환율":rate}
df=pd.DataFrame(result)
df

,외환,환율
0,USD,1159.70
1,JPY,1062.04
2,EUR,1289.18
3,CNY,167.41


#### extract() 사용해서 요소 지우기:

In [9]:
# 첫 a 태그 보기.
print(soup.find('a').prettify())

<a class="head usd" href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
 <h3 class="h_lst">
  <span class="blind">
   미국 USD
  </span>
 </h3>
 <div class="head_info point_dn">
  <span class="value">
   1,159.70
  </span>
  <span class="txt_krw">
   <span class="blind">
    원
   </span>
  </span>
  <span class="change">
   9.30
  </span>
  <span class="blind">
   하락
  </span>
 </div>
</a>



In [10]:
# 첫 a 태그 지우기.
soup.find('a').extract()

<a class="head usd" href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
<h3 class="h_lst"><span class="blind">미국 USD</span></h3>
<div class="head_info point_dn">
<span class="value">1,159.70</span>
<span class="txt_krw"><span class="blind">원</span></span>
<span class="change"> 9.30</span>
<span class="blind">하락</span>
</div>
</a>

In [11]:
# 그 다음의 a 태그 보기.
print(soup.find('a').prettify())

<a class="graph_img" href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdc', '', '', event);">
 <img alt="" height="153" src="https://ssl.pstatic.net/imgfinance/chart/marketindex/FX_USDKRW.png" width="295"/>
</a>

